# Step 5: Add a deployment pipeline
In previous five steps we implemented an automated data processing and model building pipeline. Each run of the pipeline produces a new version of the model. This notebook implements the automated model deployment step in our ML workflow.

We use again SageMaker MLOps project template to provision a ready-to use model deployment CI/CD pipeline.

This template automates the deployment of models in the SageMaker model registry to SageMaker endpoints for real-time inference. This template recognizes changes in the model registry. When a new model version is registered and approved, it automatically initiates a deployment.

![](img/six-steps-5.png)

In [6]:
import boto3
import sagemaker 

In [16]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
abalone_dataset_file_name                -> 'abalone.csv'
abalone_dataset_local_url                -> '../dataset/abalone.csv'
auto_ml_job_name                         -> 'automl-asinproc-24-14-33-56'
bucket_name                              -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                            -> 'from-idea-to-prod/xgboost'
customers_count                          -> 10000
customers_feature_group_name             -> 'fscw-customers-07-20-17-46'
data_bucket                              -> 'sagemaker-us-east-1-906545278380'
data_uploaded                            -> True
domain_id                                -> 'd-r8pbvl3oamh6'
dw_flow_file_url                         -> 's3://sagemaker-us-east-1-906545278380/feature-sto
dw_output_name                           -> '928854ec-259e-4130-8e0f-b65221b27d6e.default'
endpoint_name                            -> 'reorder-classifier-2022-07-20-19-45-09-338'
execution_role      

In [8]:
sm = boto3.client("sagemaker")

## Create an MLOps project
We follow the same procedure as in the step 4 notebook to create a model deployment MLOps project. 

### Option 1: Create project programmatically
We use `boto3` to create an MLOps project via a SageMaker API.

### Option 2: Create a project in Studio
<div class="alert alert-info"> 💡 <strong> Skip this section if you created a project programmatically </strong>

To create a project in Studio:

1. In the Studio sidebar, choose the **SageMaker resources** icon.
2. Select **Projects** from the dropdown list.
3. Choose **Create project**.
    - The **Create project** tab opens displaying a list of available templates.
4. For **SageMaker project templates**, choose **SageMaker templates**. 
5. Choose **MLOps template for model deployment**
6. Choose **Select project template**.

![](img/create-mlops-project-deploy.png)

The **Create project** tab changes to display **Project details**.
Enter the following information:
1. Project name. Note the name requirements
2. Project description, optional
3. Model package group name - enter your model package group name you used in the step 3 and 4 notebooks. It is `from-idea-to-prod-model-group` if you used the default one

Optionally, add tags, which are key-value pairs that you can use to track your projects.

![](img/project-details-deploy.png)

Choose **Create project** and wait for the project to appear in the Projects list.

## Working with MLOps project for model deployment
The template provisions a CodeCommit repository with configuration files to specify the model deployment steps, AWS CloudFormation templates to define endpoints as infrastructure, and seed code for testing the endpoint.

This template provides the following resources:

1. An AWS CodeCommit repository that contains sample code that deploys models to endpoints in staging and production environments
2. An AWS CodePipeline pipeline that has `source`, `build`, `deploy-to-staging`, and `deploy-to-production` steps. The `source` step points to the CodeCommit repository, and the `build` step gets the code from that repository and generates CloudFormation stacks to deploy. The `deploy-to-staging` and `deploy-to-production` steps deploy the CloudFormation stacks to their respective environments. There is a manual approval step between the staging and production build steps, so that a MLOps engineer must approve the model before it is deployed to production.
3. A CloudWatch event to initiate the pipeline when a model package version is approved or rejected.
4. There is also a programmatic approval step with placeholder tests in the example code in the CodeCommit repository. You can add additional tests to replace the placeholders tests.

The template also deploys an Amazon S3 bucket to store artifacts, including CodePipeline and CodeBuild artifacts, and any artifacts generated from the SageMaker pipeline runs.

The following diagram shows the architecture.

<img src="img/mlops-model-deploy.png" width="600"/>

You don't need to implement any configuration changes for the project. The model deployment pipeline works out of the box.
To start the model deployment pipeline, you must approve the model version in the model registry.

### Approve the model version
Approving the model version causes the MLOps system to launch the model deployment process. First, the model deployment pipeline deploys the model version to staging.

You can approve the model version either in Studio in the Model registry or do it programmatically in the notebook. Let's do it programatically.

In [17]:
model_package_group_name

'from-idea-to-prod-model-group'

In [12]:
# list all model packages and select the latest one
model_packages = []

for p in sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])

if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group")
    
latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

arn:aws:sagemaker:us-east-1:906545278380:model-package/from-idea-to-prod-model-group/3


The following statement sets the `ModelApprovalStatus` for the model package to `Approved`. The model package state change launches the EventBridge rule and the rule launches the CodePipeline CI/CD pipeline with model deployment.

In [13]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

### Deployment pipeline execution
The model deployment CI/CD pipeline performs the following actions:
1. Create a SageMaker endpoint with the name `<PROJECT-NAME>-staging` in the current account
2. Run the test script on the staging endpoint
3. Wait until the test result is manually approved in [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline)
4. Create a SageMaker endpoint with the name `<PROJECT-NAME>-prod` in the current account

Wait about 10-15 minutes until the pipeline deploys the staging endpoint. You can see the status of the endpoint in **SageMaker resources** > **Endpoints**:

<img src="img/endpoint-staging.png" width="400"/>

After the endpoint status changed to `InService`, you can launch the model deployment process to the production stage by manually approving the **DeployStaging** stage of the CodePipeline pipeline.

### Deploy the model version to production
Let's construct the approval link.

In [23]:
# Set to the model deployment project name
project_name = "step5-deploy-model"

# Get project id
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Construct the CodePipline pipeline name
code_pipeline_name = f"sagemaker-{project_name}-{project_id}-modeldeploy"

In [24]:
from IPython.core.display import display, HTML

# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">AWS CodePipeline</a></b>'.format(
            code_pipeline_name, region)
    )
)

Click on the link ^^^ above ^^^ to open the CodePipeline console with the pipeline execution workflow.

In the **DeployStaging stage**, choose **Review** on the **ApproveDeployment** step. In the **Review** dialog box, choose **Approve**.

Approving the **DeployStaging** stage causes the MLOps system to deploy the model to production. To view the endpoint, choose the **Endpoints** in **SageMaker resources** in Studio.

After successful completion of the CI/CD pipeline, you see two endpoints in status `InService` in **SageMaker resources** > **Endpoints**:

<img src="img/endpoint-prod.png" width="400"/>

You have just implemented a production-ready MLOps solution with the automated data processing, model building, and model deployment pipelines.

## Continue with the step 6
open the step 6 [notebook](06-monitoring.ipynb).

## Further development ideas
- Add end-to-end data encryption using AWS KMS keys
- Create a [custom SageMaker project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) for model deployment to cover your specific project requirements
- Add [multi-account model deployment](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) to your ML workflow

## Additional resources
- [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker secure MLOps](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)

# Shutdown kernel

In [5]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>